## YOLO

### Load Pretrained YOLO Model
* Model varies from `YOLO11n`, `YOLO11s`, `YOLO11m`, `YOLO11l`, `YOLO11x`. (nano, small, medium, large, xlarge.)
    - These are pretrained on COCO dataset, only for detecting 80 pre-trained classes
    - There are also models for segmentation, and pose detection.
* In this code notebook, we will going to also try the ,,Track'' mode, which is available for all detect, segment, and pose models.

In [1]:
from ultralytics import YOLO

detect_model = YOLO("yolo11n.pt")
pose_model = YOLO("yolo11n-pose.pt")

### Read your camera stream, track the objects as well as human pose

In [2]:
import numpy as np
import cv2 as cv
import copy

cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    # Use below code if your face looks blue.
    # rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    
    # Display the resulting frame
    det_res = detect_model.track(source=frame, show=False)[0]
    pos_res = pose_model.track(source=frame, show=False)[0]

    all_res = copy.deepcopy(det_res)
    all_res.keypoints = pos_res.keypoints

    res_img = all_res.plot()

    cv.imshow('result', res_img)
    k = cv.waitKey(1)

    if k == ord('q'):
        break
    
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()



0: 480x640 1 person, 49.5ms
Speed: 5.3ms preprocess, 49.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 64.8ms
Speed: 0.7ms preprocess, 64.8ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 50.1ms
Speed: 0.8ms preprocess, 50.1ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 49.9ms
Speed: 0.7ms preprocess, 49.9ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 51.4ms
Speed: 0.8ms preprocess, 51.4ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 46.7ms
Speed: 0.6ms preprocess, 46.7ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 49.4ms
Speed: 0.7ms preprocess, 49.4ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 44.8ms
Speed: 0.6ms preprocess, 44.8ms inference, 0.6ms postprocess per image at shape (1, 3, 48

### Run below cell if the opencv widget forced to be closed

In [6]:
cap.release()

## OpenAI Whisper

In [ ]:
# !pip install -U openai-whisper
# !pip install numpy==2.0
# !pip3 install pvrecorder

In [12]:
import whisper
model = whisper.load_model("tiny")

100%|█████████████████████████████████████| 72.1M/72.1M [00:05<00:00, 13.6MiB/s]
/home/hyemin/anaconda3/envs/py310/lib/python3.10/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issue

In [29]:
import struct
import wave
from pvrecorder import PvRecorder
import time

def listen(output_path='tmp.wav'):
    device_index = -1

    recorder = PvRecorder(frame_length=1024, device_index=device_index)
    recorder.start()

    wavfile = None

    if output_path is not None:
        wavfile = wave.open(output_path, "w")
        # noinspection PyTypeChecker
        wavfile.setparams((1, 2, recorder.sample_rate, recorder.frame_length, "NONE", "NONE"))

    st = time.time()
    print("=======Start Listening")
            
    while True:
        frame = recorder.read()
        if wavfile is not None:
            wavfile.writeframes(struct.pack("h" * len(frame), *frame))
        if time.time()-st > 10:
            print("=======Stopping Listening")
            break

    recorder.delete()
    if wavfile is not None:
        wavfile.close()

In [30]:
def understand(filename='tmp.wav'):
    audio = whisper.load_audio(filename)
    audio = whisper.pad_or_trim(audio)

    mel = whisper.log_mel_spectrogram(audio)
    result = whisper.decode(model, mel, whisper.DecodingOptions())

    return result.text

In [31]:
#!pip3 install ollama

In [32]:
import ollama

ollama.pull('gemma2')

{'status': 'success'}

In [ ]:
from lerobot.common.utils.utils import log_say

text = 'start'
while 'bye' not in text.lower():
    listen()
    text = understand()
    print('=======Input:', text)
    response = ollama.chat(model='gemma2', messages=[
      {
        'role': 'user',
        'content': 'Please always answer to me in 50 words. INPUT: [' + text + ']',
      },
    ])
    print('=======Output:', response['message']['content'],)
    log_say(response['message']['content'], True)
    time.sleep(7)


=======Start Listening
=======Stopping Listening
=======Input: Hello, how are you?
=======Output: I'm doing well, thanks! How about you? 😊  
=======Start Listening
=======Stopping Listening
=======Input: For me it's fine and I need to do some works.
=======Output: Sounds good! Get those works done.  👍 

Let me know if you need anything while you're at it!
=======Start Listening
=======Stopping Listening
=======Input: But I need to talk with someone because I don't want to do the work so much so I need some things
=======Output: Sounds tough. What kind of things are you needing help with? Maybe I can give you some ideas.  🤔 

=======Start Listening
=======Stopping Listening
=======Input: Maybe you can try some class materials that I need to make.
=======Output: What kind of class materials? Tell me more!  📚✏️  

=======Start Listening
=======Stopping Listening
=======Input: I need to make a slide for human robot interaction.
=======Output: Got it!  What kind of information do you want o

KeyboardInterrupt: 